In [ ]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.covidtracker.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains scripts that download data from data.gouv.fr and then process it to build many graphes.
I'm currently cleaning the code, please ask me if something is not clear enough.

The charts are exported to 'charts/images/france'.
Data is download to/imported from 'data/france'.
Requirements: please see the imports below (use pip3 to install them).

"""

In [1]:
import pandas as pd
import plotly.graph_objects as go
import france_data_management as data
from datetime import datetime
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly
import math
import os
import json
PATH = "../../"
PATH_STATS = "../../data/france/stats/"

import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [ ]:
def import_df_age():
    df = pd.read_csv(PATH+"data/france/vaccin/vacsi-a-fra.csv", sep=";")
    return df

In [10]:
df_new = pd.read_csv(PATH+"data/france/donnes-hospitalieres-covid19-nouveaux.csv", sep=";")
df_clage = pd.read_csv(PATH+"data/france/donnes-hospitalieres-clage-covid19.csv", sep=";")


In [9]:
df_new_france = df_new.groupby("jour").sum()
df_new_france.sum()

incid_hosp    437684
incid_rea      74730
incid_dc       74285
incid_rad     322389
dtype: int64

In [32]:
df_clage_france = df_clage.groupby(["jour", "cl_age90"]).sum().reset_index()
df_clage_france[df_clage_france.jour=="2021-04-12"]

,jour,cl_age90,reg,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
4290,2021-04-12,0,709,31262,5916,17532.0,7477.0,337.0,310934,73011
4291,2021-04-12,9,709,46,4,39.0,3.0,0.0,2575,4
4292,2021-04-12,19,709,103,11,68.0,6.0,18.0,2905,8
4293,2021-04-12,29,709,382,57,273.0,18.0,34.0,9842,64
4294,2021-04-12,39,709,825,187,566.0,46.0,26.0,16017,205
4295,2021-04-12,49,709,1728,489,1057.0,126.0,56.0,23684,670
4296,2021-04-12,59,709,3632,1127,2057.0,394.0,54.0,40434,2432
4297,2021-04-12,69,709,6099,1969,3106.0,953.0,71.0,53410,7610
4298,2021-04-12,79,709,7412,1681,3874.0,1804.0,53.0,61976,16167
4299,2021-04-12,89,709,7534,312,4412.0,2785.0,25.0,67143,28738


In [66]:
df = import_df_age()
df["n_dose1"] = df["n_dose1"].replace({",": ""}, regex=True).astype("int")
df = df.groupby(["clage_vacsi"]).sum()/100
df = df[1:]
df["n_dose1_pourcent"] = round(df.n_dose1/df.n_dose1.sum()*100, 1)

clage_vacsi = [24, 29, 39, 49, 59, 64, 69, 74, 79, 80]
nb_pop = [5236809, 3593713, 8034961, 8316050, 8494520, 3979481, 3801413, 3404034, 2165960, 4081928]
df_age = pd.DataFrame()
df_age["clage_vacsi"]=clage_vacsi
df_age["nb_pop"]=nb_pop

df = df.merge(df_age, left_on="clage_vacsi", right_on="clage_vacsi")
df["pop_vac"] = df["n_dose1"]/df["nb_pop"]*100
df

,clage_vacsi,n_dose1,n_dose1_pourcent,nb_pop,pop_vac
0,24,4073.0,0.4,5236809,0.077776
1,29,11120.0,1.0,3593713,0.309429
2,39,38606.0,3.4,8034961,0.480475
3,49,64746.0,5.7,8316050,0.778567
4,59,208627.0,18.5,8494520,2.456019
5,64,87428.0,7.7,3979481,2.196970
6,69,52771.0,4.7,3801413,1.388194
7,74,53858.0,4.8,3404034,1.582182
8,79,183668.0,16.2,2165960,8.479750
9,80,425719.0,37.7,4081928,10.429361


In [73]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=[str(age) + " ans" for age in df.clage_vacsi[:-1]]+["+ 80 ans"],
    y=df.pop_vac,
    text=[str(round(prct, 2)) + " %" for prct in df.pop_vac],
    textposition='auto',))
fig.update_layout(
    title={
                    'text': "% de population ayant reçu au moins 1 dose de vaccin",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    titlefont = dict(
                    size=20),
    annotations = [
                    dict(
                        x=0,
                        y=1.07,
                        xref='paper',
                        yref='paper',
                        font=dict(size=14),
                        text='{}. Données : Santé publique France. Auteur : <b>@GuillaumeRozier - covidtracker.fr.</b>'.format(datetime.strptime("2021-01-27", '%Y-%m-%d').strftime('%d %b')),                    
                        showarrow = False
                    ),
                    ]
)
fig.update_yaxes(range=[0, 100])
fig.show()

In [63]:
fig = go.Figure()
fig.add_trace(go.Pie(
    labels=[str(age) + " ans" for age in df.index[:-1]]+["+ 80 ans"],
    values=df.n_dose1_pourcent,
    text=[str(prct) + "" for prct in df.n_dose1],
    textposition='auto',))
fig.update_layout(
    title={
                    'text': "Nombre de vaccinés par tranche d'âge",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    titlefont = dict(
                    size=20),
    annotations = [
                    dict(
                        x=0,
                        y=1.07,
                        xref='paper',
                        yref='paper',
                        font=dict(size=14),
                        text='{}. Données : Santé publique France. Auteur : <b>@GuillaumeRozier - covidtracker.fr.</b>'.format(datetime.strptime("2021-01-27", '%Y-%m-%d').strftime('%d %b')),                    
                        showarrow = False
                    ),
                    ]
)
fig.show()

In [6]:
#locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
import random
import numpy as np

n_sain = 20000
x_sain = np.random.rand(1, n_sain)[0]*100
values_sain = np.random.rand(1, n_sain)[0]*100

x_az = np.random.rand(1,30)[0]*100
values_az = np.random.rand(1,30)[0]*100

fig = go.Figure()

for idx in range(len(x_sain)):
    fig.add_trace(go.Scatter(
        x=[x_sain[idx]],
        y=[values_sain[idx]],
        mode="markers",
        showlegend=False,
        marker_color="rgba(14, 201, 4, 0.5)", #"rgba(0, 0, 0, 0.5)",
        marker_size=2))

fig.add_trace(go.Scatter(
    x=x_az,
    y=values_az,
    mode="markers",
    showlegend=False,
    marker_color="rgba(201, 4, 4,0.5)", #"rgba(0, 0, 0, 0.5)",
    marker_size=2))


fig.update_yaxes(range=[0, 100], visible=False)
fig.update_xaxes(range=[0, 100], nticks=10)

fig.update_layout(
    plot_bgcolor='rgb(255,255,255)',
    title={
                'text': "Admissions en réanimation pour Covid19",
                'y':0.90,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    annotations = [
                dict(
                    x=0.5,
                    y=1.2,
                    xref='paper',
                    yref='paper',
                    text='Auteur : covidtracker.fr.'.format(),
                    showarrow = False
                )]
                 
)
fig.write_image(PATH + "images/charts/france/points_astrazeneca.jpeg", scale=4, width=800, height=350)

In [18]:
import numpy as np
np.random.rand(1,20000000)

array([0.15740955, 0.02069612, 0.61488631, ..., 0.58084254, 0.38146626,
       0.02494649])